In [22]:
#importing dependencies and setting app token
import pandas as pd
import requests
from datetime import date, datetime
from sodapy import Socrata
import csv
from io import StringIO
from sqlalchemy import create_engine

MyAppToken = 'GuILMuJLLhVOQ8u9cyXPc56p3'

#checking the day of the month and printing the result, this is used to filter the dataframe later
today = date.today()
daynum = today.strftime("%d")
day = int(daynum) - 7
print("Day =", day)

Day = 7


In [23]:
crime_data = "ijzp-q8t2"
client = Socrata("data.cityofchicago.org", MyAppToken)
df = pd.DataFrame(
    client.get(
        crime_data, 
        where=f"Date BETWEEN '2020-01-01' AND '2020-04-{day}'",
        limit=100000,
        exclude_system_fields=True
    )
)
client.close()

In [27]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year
df['date'] = pd.to_datetime(df['date']).dt.strftime('%m.%d.%Y')

df["primary_type"] = df["primary_type"].str.lower().str.title()
df["description"] = df["description"].str.lower().str.title()
df["location"] = df["location_description"].str.lower().str.title()

# Organize: 
current_df = df[["date","month", "year","primary_type", "description","latitude", "longitude", "domestic"]]
current_df.shape

(55650, 9)

In [25]:
df1 = pd.DataFrame(
    client.get(
        crime_data, 
        where=f"Date BETWEEN '2019-01-01' AND '2019-04-{day}'",
        limit=500000,
        exclude_system_fields=True
    )
)
client.close()

In [28]:
df1['month'] = pd.DatetimeIndex(df1['date']).month
df1['year'] = pd.DatetimeIndex(df1['date']).year
df1['date'] = pd.to_datetime(df1['date']).dt.strftime('%m.%d.%Y')

df1["primary_type"] = df1["primary_type"].str.lower().str.title()
df1["description"] = df1["description"].str.lower().str.title()
df1["location"] = df1["location_description"].str.lower().str.title()

# Organize: 
old_df = df1[["date","month","year","primary_type","description","latitude","longitude","domestic"]]
old_df.shape

(62515, 9)

In [45]:
dataframes = [old_df, current_df]

final_df = pd.concat(dataframes)

final_df.reset_index(inplace=True)

final_df.head()

,index,date,month,year,primary_type,description,location,latitude,longitude,domestic
0,0,01.01.2019,1,2019,Sex Offense,Aggravated Criminal Sexual Abuse,Residence,41.907072136,-87.731331357,False
1,1,01.01.2019,1,2019,Offense Involving Children,Aggravated Criminal Sexual Abuse By Family Member,Apartment,41.853079711,-87.676856787,False
2,2,01.01.2019,1,2019,Offense Involving Children,Child Abuse,Residence,41.875288057,-87.723999683,True
3,3,01.01.2019,1,2019,Deceptive Practice,Financial Identity Theft Over $ 300,Apartment,41.929013312,-87.799805351,False
4,4,01.01.2019,1,2019,Sex Offense,Agg Criminal Sexual Abuse,Apartment,41.896591951,-87.692793096,True


In [ ]:
def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

rds_connection_string = "postgres:postgres@localhost:5432/crimeandcovid19"
engine = create_engine(f'postgresql://{rds_connection_string}')
final_df.to_sql('chicago_data', engine, method=psql_insert_copy)

In [ ]:
engine.table_names()

In [ ]:
pd.read_sql_query('select * from chicago_data', con=engine).head()